In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
store_100_path = r"df_all_store_100.parquet"
store_42_path = r"df_all_store_42.parquet"
store_4906_path = r"df_all_store_4906.parquet"

In [3]:
df_100 = pd.read_parquet(store_100_path)
df_42 = pd.read_parquet(store_42_path)
df_4906 = pd.read_parquet(store_4906_path)
df_all = pd.concat([df_100, df_42, df_4906])

In [4]:
del df_100, df_42, df_4906

In [5]:
df_all['date'] = pd.to_datetime(df_all['date'])

In [6]:
warehouse_path = r"df_all_store_VZ01.parquet"
df_warehouse = pd.read_parquet(warehouse_path)

In [7]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})
df_warehouse['date'] = pd.to_datetime(df_warehouse['date'])

In [8]:
df_warehouse['sku'] = df_warehouse['sku'].astype(str)


In [9]:
df_warehouse = df_warehouse[df_warehouse['date'] > '2018-01-01']

In [10]:
df_all = df_all.merge(df_warehouse[["sku","date","warehouse_stock"]], on = ["sku","date"], how = "left")


In [11]:
df_all = df_all[df_all['date'] > '2018-01-01']

In [12]:
df_all = df_all.sort_values('date')

In [13]:
df_all = df_all.merge(df_all[df_all['stock'] > 0].groupby(['sku', 'store']).agg({'stock': 'first'}).reset_index().rename(columns={'stock':'palmers_first_allocation'}), on = ['sku', 'store'], how = 'left')

In [14]:
sku_store_df_step_g = df_all.groupby(['sku', 'store'])

In [46]:
i = 0
for store in ['42', '100']:
    for sku in ['100009161000001']:
        sku_store_df_step = sku_store_df_step_g.get_group((sku, store))
        first_stock = sku_store_df_step[sku_store_df_step["stock"]>0]["stock"].iloc[0]
        # nan in our_stock if before first stock
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "palmers_first_allocation"] = np.nan
        sku_store_df_step['days_from_first_stock'] = 1
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_first_stock"] = np.nan
        sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
        sku_store_df_step['sales_cumsum'] = sku_store_df_step['sales'].cumsum()
        sku_store_df_step['dynamic_stock_with_palmers_init'] = sku_store_df_step['palmers_first_allocation'] - sku_store_df_step['sales_cumsum']
        sku_store_df_step['sales_cumsum_divide_days_from_first_stock'] =  sku_store_df_step['sales_cumsum'] / sku_store_df_step['days_from_first_stock']
        zero_sales_days_back = 1
        sku_store_df_step['days_from_last_sales_that_not_zero'] = 1

        for index, row in sku_store_df_step.shift(1).iterrows():
            # if index is sku_store_df_step[sku_store_df_step["stock"]>0]["stock"] index
            if index > sku_store_df_step[sku_store_df_step["stock"]>0].index[0]:
                if row['sales'] == 0:
                    zero_sales_days_back += 1
                else:
                    zero_sales_days_back = 1

                sku_store_df_step.at[index, 'days_from_last_sales_that_not_zero'] = zero_sales_days_back
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_last_sales_that_not_zero"] = np.nan
        sku_store_df_step['rate_sales'] = sku_store_df_step['sales'] / sku_store_df_step['days_from_last_sales_that_not_zero']

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_32840/1219730119.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = 1
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_32840/1219730119.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_32840/1219730119.py:11: SettingWithCopyWarning: 
A v

In [47]:
print(sku_store_df_step.iloc[0:30])

                     sku store       date  average_price  \
2014680  100009161000001   100 2018-09-12            NaN   
2024532  100009161000001   100 2018-09-13            NaN   
2043874  100009161000001   100 2018-09-14            NaN   
2050162  100009161000001   100 2018-09-15            NaN   
2064786  100009161000001   100 2018-09-16            NaN   
2075427  100009161000001   100 2018-09-17            NaN   
2088404  100009161000001   100 2018-09-18            NaN   
2092978  100009161000001   100 2018-09-19            NaN   
2104613  100009161000001   100 2018-09-20            NaN   
2124014  100009161000001   100 2018-09-21            NaN   
2129414  100009161000001   100 2018-09-22            NaN   
2144075  100009161000001   100 2018-09-23            NaN   
2158493  100009161000001   100 2018-09-24            NaN   
2166636  100009161000001   100 2018-09-25            NaN   
2173706  100009161000001   100 2018-09-26            NaN   
2187204  100009161000001   100 2018-09-2

In [ ]:
sku_store_df_step.loc[sku_store_df_step['date'] < sku_store_df_step[sku_store_df_step["stock"]>0]['date'].iloc[0], "sum_of_our_allocation_stock"] = np.nan

In [ ]:
sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]]

In [ ]:
print(df_all[(df_all['sku'] == '100009161000001') & (df_all['store'] == '100')])